In [9]:
!pip install transformers datasets sacrebleu sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.7 MB/s eta 0:00:00


In [10]:
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
import torch


In [13]:
from datasets import load_dataset

# Try loading the opus_books dataset with a safer approach
try:
    dataset = load_dataset("opus_books", "en_ru")
    print("Dataset loaded successfully!")
except Exception as e:
    print(f"Error loading dataset: {e}")


Error loading dataset: Invalid pattern: '**' can only be an entire path component


In [15]:
from datasets import load_dataset

# Try loading the opus_books dataset with a safer approach
try:
    dataset = load_dataset("opus_books", "en_ru")
    print("Dataset loaded successfully!")
    # Selecting the first 1000 training samples and 200 validation samples
    train_data = dataset["train"].select(range(1000))  # 1000 samples for training
    val_data = dataset["validation"].select(range(200))  # 200 samples for validation

    # Check the structure of the selected subsets
    print(f"Training data sample size: {len(train_data)}")
    print(f"Validation data sample size: {len(val_data)}")

except Exception as e:
    print(f"Error loading dataset: {e}")

Error loading dataset: Invalid pattern: '**' can only be an entire path component


In [16]:
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
import torch



In [21]:
# Load the Marian tokenizer
model_name = "Helsinki-NLP/opus-mt-en-ru"
tokenizer = MarianTokenizer.from_pretrained(model_name)


In [22]:
def tokenize(example):
    # Extract the first translation entry if it's a list (handling potential multiple translations)
    translation = example["translation"][0] if isinstance(example["translation"], list) else example["translation"]
    # Tokenizing the English and Russian translations
    inputs = tokenizer(translation["en"], truncation=True, padding="max_length", max_length=128)
    targets = tokenizer(translation["ru"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]  # Add the Russian tokens as labels
    return inputs

In [23]:
def tokenize(examples):
    # Extract translations (handling potential multiple translations)
    translations = [ex[0] if isinstance(ex, list) else ex for ex in examples["translation"]]

    # Prepare English translations for tokenization
    en_texts = [t["en"] for t in translations]

    # Prepare Russian translations for tokenization
    ru_texts = [t["ru"] for t in translations]

    # Tokenize English translations
    inputs = tokenizer(en_texts, truncation=True, padding="max_length", max_length=128, return_tensors="pt")

    # Tokenize Russian translations
    targets = tokenizer(ru_texts, truncation=True, padding="max_length", max_length=128, return_tensors="pt")

    # Add the Russian tokens as labels
    inputs["labels"] = targets["input_ids"]

    return inputs

In [26]:
# Load the pre-trained MarianMT model
model = MarianMTModel.from_pretrained(model_name)


In [28]:
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments


In [ ]:
import transformers
print(transformers.__version__)


4.51.3


In [30]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",          # Where to save the model and logs
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    logging_dir="./logs",            # Directory for logs
    logging_steps=500,               # Log every 500 steps
    save_steps=1000,                 # Save model every 1000 steps
    load_best_model_at_end=False,    # Disable loading the best model
    weight_decay=0.01,               # Weight decay
)

In [35]:
from transformers import MarianMTModel, MarianTokenizer, pipeline

# Load the fine-tuned model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-ru"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Create a translation pipeline
translator = pipeline("translation_en_to_ru", model=model, tokenizer=tokenizer)

# Example English sentences
english_sentences = ["My name is Nemat Dadashov"]



# Translate
russian_translations = translator(english_sentences)

# Display the results
for en, ru in zip(english_sentences, russian_translations):
    print(f"ENGLISH: {en}")
    print(f"RUSSIAN: {ru['translation_text']}\n")


Device set to use cuda:0


ENGLISH: My name is Nemat Dadashov
RUSSIAN: Меня зовут Немат Дадашов

